In [0]:
%fs ls /Volumes/dev/demo_db/landing

### Ingest Data using Spark Streaming API

In [0]:
def ingest():
    source_df = (spark.readStream
                 .format("cloudFiles")
                 .option("cloudFiles.format", "csv")
                 .option("header","true")
                 .option("timestampFormat","d-M-y H:m:s")
                 .option("cloudFiles.inferColumnTypes", "true")
                 .option("cloudFiles.schemaLocation", "/Volumes/dev/demo_db/landing/checkpoint/invoice_final_schema")
                 .option("cloudFiles.schemaHints", "InvoiceNo string, CustomerID string")
                 .load("/Volumes/dev/demo_db/landing")
                 )
    
    write_query = (source_df.writeStream
                   .format("delta")
                   .option("checkpointLocation", "/Volumes/dev/demo_db/landing/checkpoint/invoice_final2")
                   .option("mergeSchema","true")
                   .outputMode("append")
                   .trigger(availableNow = True)
                   .toTable("dev.demo_db.invoices_raw_5")
                  )

ingest()


In [0]:
def ingest():
    source_df = (
        spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "csv")
        .option("header", "true")
        .option("timestampFormat", "d-M-y H:m:s")
        .option("cloudFiles.inferColumnTypes", "true")
        .option("cloudFiles.schemaLocation", "/Volumes/dev/demo_db/landing/checkpoint/invoice_final_schema")
        .option("cloudFiles.schemaHints", "InvoiceNo string, CustomerID string")
        .load("/Volumes/dev/demo_db/landing")
    )

    # Sanitize column names: replace invalid characters with underscore
    for c in source_df.columns:
        new_name = (
            c.replace(" ", "_")
            .replace(",", "_")
            .replace(";", "_")
            .replace("{", "_")
            .replace("}", "_")
            .replace("(", "_")
            .replace(")", "_")
            .replace("\n", "_")
            .replace("\t", "_")
            .replace("=", "_")
        )
        if new_name != c:
            source_df = source_df.withColumnRenamed(c, new_name)

    (source_df.writeStream
        .format("delta")
        .option("checkpointLocation", "/Volumes/dev/demo_db/landing/checkpoint/invoice_final2")
        .option("mergeSchema", "true")
        .outputMode("append")
        .trigger(availableNow=True)
        .toTable("dev.demo_db.invoices_raw_5")
    )

ingest()

In [0]:
%sql
select * from dev.demo_db.invoices_raw_5

In [0]:
%sql
describe dev.demo_db.invoices_raw_5


In [0]:
%fs cp /Volumes/dev/demo_db/landing/invoices/invoices_2021.csv /Volumes/dev/demo_db/landing

In [0]:
ingest()

In [0]:
%sql
select * from dev.demo_db.invoices_raw_5